In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession.\
builder.\
config('spark.ui.port','0').\
config("spark.sql.warehouse.dir",f"/user/itv012676/warehouse").\
enableHiveSupport().\
master('yarn').\
getOrCreate()

### 1.1.Create a managed spark table and load data to it from the given csv file
  

In [2]:
spark.sql("create database if not exists week5_assignment_itv012676")

""


In [3]:
spark.sql("use week5_assignment_itv012676")

""


In [4]:
spark.sql("show tables").show()

+--------------------+--------------------+-----------+
|            database|           tableName|isTemporary|
+--------------------+--------------------+-----------+
|week5_assignment_...|customers_itv0126...|      false|
|week5_assignment_...| groceries_itv012676|      false|
|week5_assignment_...|groceries_itv0126...|      false|
|week5_assignment_...|products_itv012676_m|      false|
+--------------------+--------------------+-----------+



In [5]:
spark.sql("create table week5_assignment_itv012676.groceries_itv012676_m(order_id string, location string, item string, order_date string, quantity integer)")

""


In [6]:
spark.sql("show tables").show(truncate=False)

+--------------------------+---------------------------+-----------+
|database                  |tableName                  |isTemporary|
+--------------------------+---------------------------+-----------+
|week5_assignment_itv012676|customers_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676        |false      |
|week5_assignment_itv012676|groceries_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676_managed|false      |
|week5_assignment_itv012676|products_itv012676_m       |false      |
+--------------------------+---------------------------+-----------+



In [7]:
groceries_df = spark.read \
.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("/public/trendytech/groceries.csv")

In [8]:
groceries_df.show(5)

+--------+--------+--------+----------+--------+
|order_id|location|    item|order_date|quantity|
+--------+--------+--------+----------+--------+
|      o1| Seattle| Bananas|01/01/2017|       7|
|      o2|    Kent|  Apples|02/01/2017|      20|
|      o3|Bellevue| Flowers|02/01/2017|      10|
|      o4| Redmond|    Meat|03/01/2017|      40|
|      o5| Seattle|Potatoes|04/01/2017|       9|
+--------+--------+--------+----------+--------+
only showing top 5 rows



In [9]:
groceries_df.createOrReplaceTempView("groceries_data")

In [10]:
spark.sql("select * from groceries_data").show()

+--------+---------+--------+----------+--------+
|order_id| location|    item|order_date|quantity|
+--------+---------+--------+----------+--------+
|      o1|  Seattle| Bananas|01/01/2017|       7|
|      o2|     Kent|  Apples|02/01/2017|      20|
|      o3| Bellevue| Flowers|02/01/2017|      10|
|      o4|  Redmond|    Meat|03/01/2017|      40|
|      o5|  Seattle|Potatoes|04/01/2017|       9|
|      o6| Bellevue|   Bread|04/01/2017|       5|
|      o7|  Redmond|   Bread|05/01/2017|       5|
|      o8| Issaquah|   Onion|05/01/2017|       4|
|      o9|  Redmond|  Cheese|05/01/2017|      15|
|     o10| Issaquah|   Onion|06/01/2017|       4|
|     o11|   Renton|   Bread|05/01/2017|       5|
|     o12| Issaquah|   Onion|07/01/2017|       4|
|     o13|Sammamish|   Bread|07/01/2017|       5|
|     o14| Issaquah|  Tomato|07/01/2017|       6|
|     o15| Issaquah|    Meat|08/01/2017|       3|
|     o16| Issaquah|    Meat|09/01/2017|       5|
|     o17| Issaquah|    Meat|10/01/2017|       6|


In [11]:
spark.sql("insert into week5_assignment_itv012676.groceries_itv012676_m select * from groceries_data")

""


### 1.2. Create an external spark table with the same data.

In [12]:
spark.sql("create table week5_assignment_itv012676.groceries_itv012676_e(order_id string, location string, item string, order_date string, quantity integer) using csv location '/public/trendytech/groceries.csv'")

""


In [13]:
spark.sql("show tables").show(truncate=False)

+--------------------------+---------------------------+-----------+
|database                  |tableName                  |isTemporary|
+--------------------------+---------------------------+-----------+
|week5_assignment_itv012676|customers_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676        |false      |
|week5_assignment_itv012676|groceries_itv012676_e      |false      |
|week5_assignment_itv012676|groceries_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676_managed|false      |
|week5_assignment_itv012676|products_itv012676_m       |false      |
|                          |groceries_data             |true       |
+--------------------------+---------------------------+-----------+



### 1.3. Verify that the data has been successfully loaded into both the tables

In [14]:
spark.sql("select * from week5_assignment_itv012676.groceries_itv012676_m").show(10)

+--------+--------+--------+----------+--------+
|order_id|location|    item|order_date|quantity|
+--------+--------+--------+----------+--------+
|      o1| Seattle| Bananas|01/01/2017|       7|
|      o2|    Kent|  Apples|02/01/2017|      20|
|      o3|Bellevue| Flowers|02/01/2017|      10|
|      o4| Redmond|    Meat|03/01/2017|      40|
|      o5| Seattle|Potatoes|04/01/2017|       9|
|      o6|Bellevue|   Bread|04/01/2017|       5|
|      o7| Redmond|   Bread|05/01/2017|       5|
|      o8|Issaquah|   Onion|05/01/2017|       4|
|      o9| Redmond|  Cheese|05/01/2017|      15|
|     o10|Issaquah|   Onion|06/01/2017|       4|
+--------+--------+--------+----------+--------+
only showing top 10 rows



In [15]:
spark.sql("select * from week5_assignment_itv012676.groceries_itv012676_e").show(10)

+--------+--------+--------+----------+--------+
|order_id|location|    item|order_date|quantity|
+--------+--------+--------+----------+--------+
|order_id|location|    item|order_date|    null|
|      o1| Seattle| Bananas|01/01/2017|       7|
|      o2|    Kent|  Apples|02/01/2017|      20|
|      o3|Bellevue| Flowers|02/01/2017|      10|
|      o4| Redmond|    Meat|03/01/2017|      40|
|      o5| Seattle|Potatoes|04/01/2017|       9|
|      o6|Bellevue|   Bread|04/01/2017|       5|
|      o7| Redmond|   Bread|05/01/2017|       5|
|      o8|Issaquah|   Onion|05/01/2017|       4|
|      o9| Redmond|  Cheese|05/01/2017|      15|
+--------+--------+--------+----------+--------+
only showing top 10 rows



### 1.4 Drop the managed and external tables and see the differences

In [16]:
spark.sql("drop table week5_assignment_itv012676.groceries_itv012676_e") #dropping external table

""


In [17]:
spark.sql("show tables").show(truncate = False)

+--------------------------+---------------------------+-----------+
|database                  |tableName                  |isTemporary|
+--------------------------+---------------------------+-----------+
|week5_assignment_itv012676|customers_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676        |false      |
|week5_assignment_itv012676|groceries_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676_managed|false      |
|week5_assignment_itv012676|products_itv012676_m       |false      |
|                          |groceries_data             |true       |
+--------------------------+---------------------------+-----------+



In [18]:
spark.sql("drop table week5_assignment_itv012676.groceries_itv012676_m") #dropping managed table

""


In [19]:
spark.sql("show tables").show(truncate = False)

+--------------------------+---------------------------+-----------+
|database                  |tableName                  |isTemporary|
+--------------------------+---------------------------+-----------+
|week5_assignment_itv012676|customers_itv012676_m      |false      |
|week5_assignment_itv012676|groceries_itv012676        |false      |
|week5_assignment_itv012676|groceries_itv012676_managed|false      |
|week5_assignment_itv012676|products_itv012676_m       |false      |
|                          |groceries_data             |true       |
+--------------------------+---------------------------+-----------+



### Repeating Above all with Json File

In [20]:
spark.sql("create database if not exists week5_assignment_itv012676_2")

""


In [21]:
spark.sql("use week5_assignment_itv012676_2")

""


In [22]:
spark.sql("show tables").show()

+--------+--------------+-----------+
|database|     tableName|isTemporary|
+--------+--------------+-----------+
|        |groceries_data|       true|
+--------+--------------+-----------+



In [23]:
spark.sql("create table week5_assignment_itv012676_2.groceries_itv012676_m(customer_id string, order_date string, order_id string, order_status string)")

""


In [24]:
spark.sql("show tables").show(truncate=False)

+----------------------------+---------------------+-----------+
|database                    |tableName            |isTemporary|
+----------------------------+---------------------+-----------+
|week5_assignment_itv012676_2|groceries_itv012676_m|false      |
|                            |groceries_data       |true       |
+----------------------------+---------------------+-----------+



In [25]:
groceries_df = spark.read \
.format("json") \
.option("header","true") \
.option("inferSchema","true") \
.load("/public/trendytech/orders_wh.json/part-00000-68544d18-9a34-443f-bf0e-1dd8103ff94e-c000.json")

In [26]:
groceries_df.show(5,truncate=False)

+-----------+---------------------+--------+---------------+
|customer_id|order_date           |order_id|order_status   |
+-----------+---------------------+--------+---------------+
|11599      |2013-07-25 00:00:00.0|1       |CLOSED         |
|256        |2013-07-25 00:00:00.0|2       |PENDING_PAYMENT|
|12111      |2013-07-25 00:00:00.0|3       |COMPLETE       |
|8827       |2013-07-25 00:00:00.0|4       |CLOSED         |
|11318      |2013-07-25 00:00:00.0|5       |COMPLETE       |
+-----------+---------------------+--------+---------------+
only showing top 5 rows



In [27]:
groceries_df.createOrReplaceTempView("groceries_data")

In [28]:
spark.sql("select * from groceries_data").show(truncate = False)

+-----------+---------------------+--------+---------------+
|customer_id|order_date           |order_id|order_status   |
+-----------+---------------------+--------+---------------+
|11599      |2013-07-25 00:00:00.0|1       |CLOSED         |
|256        |2013-07-25 00:00:00.0|2       |PENDING_PAYMENT|
|12111      |2013-07-25 00:00:00.0|3       |COMPLETE       |
|8827       |2013-07-25 00:00:00.0|4       |CLOSED         |
|11318      |2013-07-25 00:00:00.0|5       |COMPLETE       |
|7130       |2013-07-25 00:00:00.0|6       |COMPLETE       |
|4530       |2013-07-25 00:00:00.0|7       |COMPLETE       |
|2911       |2013-07-25 00:00:00.0|8       |PROCESSING     |
|5657       |2013-07-25 00:00:00.0|9       |PENDING_PAYMENT|
|5648       |2013-07-25 00:00:00.0|10      |PENDING_PAYMENT|
|918        |2013-07-25 00:00:00.0|11      |PAYMENT_REVIEW |
|1837       |2013-07-25 00:00:00.0|12      |CLOSED         |
|9149       |2013-07-25 00:00:00.0|13      |PENDING_PAYMENT|
|9842       |2013-07-25 

In [29]:
spark.sql("insert into week5_assignment_itv012676_2.groceries_itv012676_m select * from groceries_data")

""


In [30]:
spark.sql("create table week5_assignment_itv012676_2.groceries_itv012676_e(customer_id string, order_date string, order_id string, order_status string) using json location '/public/trendytech/orders_wh.json/part-00000-68544d18-9a34-443f-bf0e-1dd8103ff94e-c000.json'")

""


In [31]:
spark.sql("show tables").show(truncate=False)

+----------------------------+---------------------+-----------+
|database                    |tableName            |isTemporary|
+----------------------------+---------------------+-----------+
|week5_assignment_itv012676_2|groceries_itv012676_e|false      |
|week5_assignment_itv012676_2|groceries_itv012676_m|false      |
|                            |groceries_data       |true       |
+----------------------------+---------------------+-----------+



In [32]:
spark.sql("select * from week5_assignment_itv012676_2.groceries_itv012676_m").show(10)

+-----------+--------------------+--------+---------------+
|customer_id|          order_date|order_id|   order_status|
+-----------+--------------------+--------+---------------+
|      11599|2013-07-25 00:00:...|       1|         CLOSED|
|        256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|      12111|2013-07-25 00:00:...|       3|       COMPLETE|
|       8827|2013-07-25 00:00:...|       4|         CLOSED|
|      11318|2013-07-25 00:00:...|       5|       COMPLETE|
|       7130|2013-07-25 00:00:...|       6|       COMPLETE|
|       4530|2013-07-25 00:00:...|       7|       COMPLETE|
|       2911|2013-07-25 00:00:...|       8|     PROCESSING|
|       5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|       5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
+-----------+--------------------+--------+---------------+
only showing top 10 rows



In [33]:
spark.sql("select * from week5_assignment_itv012676_2.groceries_itv012676_e").show(10)

+-----------+--------------------+--------+---------------+
|customer_id|          order_date|order_id|   order_status|
+-----------+--------------------+--------+---------------+
|      11599|2013-07-25 00:00:...|       1|         CLOSED|
|        256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|      12111|2013-07-25 00:00:...|       3|       COMPLETE|
|       8827|2013-07-25 00:00:...|       4|         CLOSED|
|      11318|2013-07-25 00:00:...|       5|       COMPLETE|
|       7130|2013-07-25 00:00:...|       6|       COMPLETE|
|       4530|2013-07-25 00:00:...|       7|       COMPLETE|
|       2911|2013-07-25 00:00:...|       8|     PROCESSING|
|       5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|       5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
+-----------+--------------------+--------+---------------+
only showing top 10 rows



In [34]:
spark.sql("drop table week5_assignment_itv012676_2.groceries_itv012676_e") #dropping external table

""


In [35]:
spark.sql("drop table week5_assignment_itv012676_2.groceries_itv012676_m") #dropping managed table

""


### 2.1.Find the total number of products in the given dataset.

In [36]:
products_df = spark.read \
.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("products_data/products_updated.csv")

In [37]:
products_df.show()

+----------+--------+--------------------+-----------+------+--------------------+
|product_id|category|        product_name|description| price|           image_url|
+----------+--------+--------------------+-----------+------+--------------------+
|         1|       2|Quest Q64 10 FT. ...|       null| 59.98|http://images.acm...|
|         2|       2|Under Armour Men'...|       null|129.99|http://images.acm...|
|         3|       2|Under Armour Men'...|       null| 89.99|http://images.acm...|
|         4|       2|Under Armour Men'...|       null| 89.99|http://images.acm...|
|         5|       2|Riddell Youth Rev...|       null|199.99|http://images.acm...|
|         6|       2|Jordan Men's VI R...|       null|134.99|http://images.acm...|
|         7|       2|Schutt Youth Recr...|       null| 99.99|http://images.acm...|
|         8|       2|Nike Men's Vapor ...|       null|129.99|http://images.acm...|
|         9|       2|Nike Adult Vapor ...|       null|  50.0|http://images.acm...|
|   

In [38]:
total_products = products_df.count()
print(total_products)

1345


In [41]:
# spark.sql("drop table week5_assignment_itv012676.products_itv012676_m")

""


In [42]:
spark.sql("create table week5_assignment_itv012676.products_itv012676_m(product_id integer, category integer,product_name string, description string, price float, image_url string)")

""


In [43]:
products_df.createOrReplaceTempView("products_data")

In [44]:
spark.sql("insert into week5_assignment_itv012676.products_itv012676_m select * from products_data")

""


In [45]:
spark.sql("select * from week5_assignment_itv012676.products_itv012676_m").show(10,truncate=False)

+----------+--------+---------------------------------------------+-----------+------+---------------------------------------------------------------------------------------+
|product_id|category|product_name                                 |description|price |image_url                                                                              |
+----------+--------+---------------------------------------------+-----------+------+---------------------------------------------------------------------------------------+
|1         |2       |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|null       |59.98 |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy  |
|2         |2       |Under Armour Men's Highlight MC Football Clea|null       |129.99|http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat       |
|3         |2       |Under Armour Men's Renegade D Mid Football Cl|null       |89.99 |http://images.acmesports.sports/Under+A

In [46]:
spark.sql("select COUNT(*) as total_products from week5_assignment_itv012676.products_itv012676_m")

total_products
1345


### 2.2 Find the number of unique categories of products in the given dataset

In [47]:
unique_categories = products_df.select("category").distinct().count()
unique_categories

55

In [48]:
spark.sql("select  COUNT(distinct category) as distinct_categories from week5_assignment_itv012676.products_itv012676_m")

distinct_categories
55


### 2.3 Find the top 5 most expensive products based on their price ,along with their product name,category,and imageURL

In [49]:
top_5_expensive_products = products_df.select("product_name","category","price","image_url").orderBy("price",ascending=False).limit(5)
top_5_expensive_products.show(truncate=False)

+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|product_name                                    |category|price  |image_url                                                                          |
+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|SOLE E35 Elliptical                             |10      |1999.99|http://images.acmesports.sports/SOLE+E35+Elliptical                                |
|SOLE F85 Treadmill                              |4       |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |10      |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |22      |1799.99|http://images.acmespo

In [50]:
spark.sql("select product_name,category,price,image_url from week5_assignment_itv012676.products_itv012676_m ORDER BY price DESC limit 5").show(truncate = False)

+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|product_name                                    |category|price  |image_url                                                                          |
+------------------------------------------------+--------+-------+-----------------------------------------------------------------------------------+
|SOLE E35 Elliptical                             |10      |1999.99|http://images.acmesports.sports/SOLE+E35+Elliptical                                |
|SOLE F85 Treadmill                              |4       |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |10      |1799.99|http://images.acmesports.sports/SOLE+F85+Treadmill                                 |
|SOLE F85 Treadmill                              |22      |1799.99|http://images.acmespo

### 3.1 Find the total number of customers in each state.

In [51]:
customers_df = spark.read \
.format("csv") \
.option("header","true") \
.option("inferSchema","true") \
.load("/user/itv012676/customers_data/customers_updated.csv")

In [52]:
customers_df.show()

+-----------+-------------------+------------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_first_name|customer_last_name|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+-------------------+------------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|            Richard|         Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|               Mary|           Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|                Ann|             Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          4|               Mary|             Jones|     XXXXXXXXX|        XXXXXXX

In [54]:
# spark.sql("drop table week5_assignment_itv012676.customers_itv012676_m")
spark.sql("create table week5_assignment_itv012676.customers_itv012676_m(customer_id integer, customer_first_name string, customer_last_name string, customer_email string, customer_password string, customer_street string, customer_city string, customer_state string, customer_zipcode integer)")
customers_df.createOrReplaceTempView("customers_data")
spark.sql("insert into week5_assignment_itv012676.customers_itv012676_m select * from customers_data")
spark.sql("select * from week5_assignment_itv012676.customers_itv012676_m").show(10,truncate=False)

+-----------+-------------------+------------------+--------------+-----------------+---------------------------+-------------+--------------+----------------+
|customer_id|customer_first_name|customer_last_name|customer_email|customer_password|customer_street            |customer_city|customer_state|customer_zipcode|
+-----------+-------------------+------------------+--------------+-----------------+---------------------------+-------------+--------------+----------------+
|1          |Richard            |Hernandez         |XXXXXXXXX     |XXXXXXXXX        |6303 Heather Plaza         |Brownsville  |TX            |78521           |
|2          |Mary               |Barrett           |XXXXXXXXX     |XXXXXXXXX        |9526 Noble Embers Ridge    |Littleton    |CO            |80126           |
|3          |Ann                |Smith             |XXXXXXXXX     |XXXXXXXXX        |3422 Blue Pioneer Bend     |Caguas       |PR            |725             |
|4          |Mary               |Jones  

In [55]:
customers_df.groupBy("customer_state").count()

customer_state,count
AZ,213
SC,41
LA,63
MN,39
NJ,219
DC,42
OR,119
VA,136
RI,15
KY,35


In [56]:
spark.sql("select customer_state,count(*) as count from week5_assignment_itv012676.customers_itv012676_m GROUP BY customer_state")

customer_state,count
AZ,213
SC,41
LA,63
MN,39
NJ,219
DC,42
OR,119
VA,136
RI,15
KY,35


### 3.2 Find the top 5 most common last names among the customers.

In [57]:
customers_df.select("customer_last_name").groupBy("customer_last_name").count().orderBy("count",ascending=False).show(5)

+------------------+-----+
|customer_last_name|count|
+------------------+-----+
|             Smith| 4626|
|           Johnson|   76|
|          Williams|   69|
|             Jones|   65|
|             Brown|   62|
+------------------+-----+
only showing top 5 rows



In [59]:
spark.sql("select customer_last_name,count(*) as count from week5_assignment_itv012676.customers_itv012676_m GROUP BY customer_last_name ORDER BY count DESC LIMIT 5")

customer_last_name,count
Smith,4626
Johnson,76
Williams,69
Jones,65
Brown,62


### 3.3 Check whether there are any customers whose zip codes are not valid (i.e., not equal to 5 digits).

In [75]:
from pyspark.sql.functions import length
invalid_zipcodes = customers_df.filter(length("customer_zipcode")!=5)
if invalid_zipcodes.count() == 0:
    print("There are no customers with invalid zip codes...")
else:
    print("These are the customers with invalid zip codes ")
    invalid_zipcodes.show()

These are the customers with invalid zip codes 
+-----------+-------------------+------------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_first_name|customer_last_name|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+-------------------+------------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          3|                Ann|             Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
|          5|             Robert|            Hudson|     XXXXXXXXX|        XXXXXXXXX|10 Crystal River ...|       Caguas|            PR|             725|
|          6|               Mary|             Smith|     XXXXXXXXX|        XXXXXXXXX|3151 Sleepy Quail...|      Passaic|            NJ|            7055|
|          7|            Melissa| 

In [76]:
spark.sql("Select * from week5_assignment_itv012676.customers_itv012676_m where LENGTH(customer_zipcode)!=5 ")

customer_id,customer_first_name,customer_last_name,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer...,Caguas,PR,725
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River ...,Caguas,PR,725
6,Mary,Smith,XXXXXXXXX,XXXXXXXXX,3151 Sleepy Quail...,Passaic,NJ,7055
7,Melissa,Wilcox,XXXXXXXXX,XXXXXXXXX,9453 High Concession,Caguas,PR,725
8,Megan,Smith,XXXXXXXXX,XXXXXXXXX,3047 Foggy Forest...,Lawrence,MA,1841
9,Mary,Perez,XXXXXXXXX,XXXXXXXXX,3616 Quaking Street,Caguas,PR,725
11,Mary,Huffman,XXXXXXXXX,XXXXXXXXX,3169 Stony Woods,Caguas,PR,725
13,Mary,Baldwin,XXXXXXXXX,XXXXXXXXX,7922 Iron Oak Gar...,Caguas,PR,725
16,Tiffany,Smith,XXXXXXXXX,XXXXXXXXX,6651 Iron Port,Caguas,PR,725
19,Stephanie,Mitchell,XXXXXXXXX,XXXXXXXXX,3543 Red Treasure...,Caguas,PR,725


### 3.4 Count the number of customers who have valid zip codes.

In [77]:
customers_df.filter(length("customer_zipcode")==5).count()

7244

In [80]:
spark.sql("select count(*) as valid_zipcodes_count from week5_assignment_itv012676.customers_itv012676_m WHERE LENGTH(customer_zipcode)==5")

valid_zipcodes_count
7244


### 3.5 Find the number of customers from each city in the state of California(CA).

In [83]:
customers_df.filter("customer_state == 'CA'").groupBy("customer_city").count()

customer_city,count
Corona,14
Pittsburg,4
Compton,19
Palo Alto,6
Hanford,9
Anaheim,19
Folsom,6
Napa,8
Temecula,6
Reseda,6


In [85]:
spark.sql("select customer_city,count(*) as count from week5_assignment_itv012676.customers_itv012676_m WHERE customer_state == 'CA' GROUP BY customer_city")

customer_city,count
Corona,14
Pittsburg,4
Compton,19
Palo Alto,6
Hanford,9
Anaheim,19
Folsom,6
Napa,8
Temecula,6
Reseda,6
